In [ ]:
!pip install yt-dlp
!pip install firebase-admin

In [ ]:
from yt_dlp import YoutubeDL
from datetime import date
import json
import os
import re

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# firebase/firestore

# Uploaded service account key file
cred = credentials.Certificate(userdata.get('FIREBASE_CERT_PATH'))

# Initialize the Firebase Admin SDK if it hasn't been initialized yet
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)

# Get a Firestore client
db = firestore.client()

print("Firebase Admin SDK initialized and Firestore client obtained!")

In [ ]:
favorite_music_url = 'https://www.youtube.com/playlist?list=PL2E-fKOunybIoCAccVG2flYZSmltOymBa'
playlist_url = 'https://www.youtube.com/playlist?list=PL2E-fKOunybLmPf--AqaUFZVVtrfIpNXi'
col = db.collection('playlist')

In [ ]:
def get_playlist_entries(playlist_url):
    # get info on playlist
    # ℹ️ See help(YoutubeDL) for a list of available options and public functions
    ydl_opts = {'extract_flat': True}
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(playlist_url, download=False)

        # ℹ️ ydl.sanitize_info makes the info json-serializable
        info = json.dumps(ydl.sanitize_info(info))

    info_dict = json.loads(info)
    entries = info_dict['entries']

    return entries

In [ ]:
def save_playlist_data():
    playlist = get_playlist_entries(playlist_url)
    favorites = get_playlist_entries(favorite_music_url)
    favorites = set([i['id'] for i in favorites])
    today = date.today().strftime("%d/%m/%Y")

    for video in playlist:
        video_id = video['id']
        # check if on favorites list
        favorite = video_id in favorites

        # check if already on list
        doc = col.document(video_id).get()

        if doc.exists:
            # if on list check if video became unavailable
            if video['duration'] == None:
                col.document(video_id).update({'removedFromYT': today})
                col.document(video_id).update({'lastUpdated': today})
            else:
                col.document(video_id).update({'lastUpdated': today})
        else:
            # skip if the video info cannot be salvaged
            if video['duration'] == None:
                continue

            data = {
                'videoId': video_id,
                'title': video['title'],
                'channel': video['channel'],
                'favorite': favorite,

                'addedToList': today,
                'lastUpdated': today,

                'removedFromYT': None,
            }

            # add to list
            col.document(video_id).set(data)

    return

In [ ]:
save_playlist_data()